# 🚀 Story Video Generator - FULL GPU Server

**Complete Video Generation on GPU!**

**Features**:
- 🎤 Kokoro TTS (48 voices)
- 🎨 SDXL-Turbo (AI images 16:9)
- 🎬 FFmpeg (video compilation with ALL effects)
- ⚡ GPU-accelerated everything
- 🌐 Ngrok public URL

**Requirements**: 
- Runtime: GPU (T4, V100, A100)
- GPU RAM: 15+ GB

---

## ⚠️ IMPORTANT: Enable GPU Runtime

1. Click: `Runtime` → `Change runtime type`
2. Select: `Hardware accelerator` → `GPU` → `T4 GPU`
3. Click: `Save`
4. Run all cells

---

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 📦 STEP 1: INSTALL DEPENDENCIES
# ═══════════════════════════════════════════════════════════════════════════════

print("📦 Installing dependencies...\n")

# ✅ FFmpeg (for video processing)
print("🎬 Installing FFmpeg...")
!apt-get update -qq
!apt-get install -y -qq ffmpeg
!ffmpeg -version | head -n 1
print("   ✅ FFmpeg installed!\n")

# Core dependencies
%pip install -q --ignore-installed blinker flask flask-cors pyngrok

# Kokoro TTS
%pip install -q git+https://github.com/thewh1teagle/kokoro-onnx.git
%pip install -q soundfile numpy scipy onnxruntime

# SDXL-Turbo
%pip install -q diffusers transformers accelerate safetensors

# Image/Video processing
%pip install -q pillow opencv-python-headless

# Torch (GPU support)
%pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

print("\n✅ All dependencies installed!")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🔧 STEP 2: SETUP GPU & IMPORTS
# ═══════════════════════════════════════════════════════════════════════════════

try:
    from flask import Flask
    from flask_cors import CORS
except ImportError:
    %pip install -q --ignore-installed blinker flask flask-cors pyngrok
    from flask import Flask
    from flask_cors import CORS

import os
import gc
import torch
import json
import subprocess
import base64
from flask import request, jsonify, send_file
from pyngrok import ngrok
from pathlib import Path
from threading import Thread
from PIL import Image
import io

# GPU Detection
if torch.cuda.is_available():
    device = 'cuda'
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"✅ GPU ENABLED: {gpu_name}")
    print(f"💾 GPU Memory: {gpu_memory:.1f} GB")
    print(f"🔥 CUDA Version: {torch.version.cuda}")
else:
    device = 'cpu'
    print("⚠️  WARNING: GPU NOT DETECTED")

print(f"\n🚀 Device: {device}")

# Create output directory
output_dir = Path('/content/outputs')
output_dir.mkdir(exist_ok=True)

print(f"📁 Output directory: {output_dir}")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🧠 STEP 3: MEMORY MANAGEMENT
# ═══════════════════════════════════════════════════════════════════════════════

tts_pipeline = None
img_pipeline = None

def clear_gpu_memory():
    global tts_pipeline, img_pipeline
    if tts_pipeline is not None:
        del tts_pipeline
        tts_pipeline = None
    if img_pipeline is not None:
        del img_pipeline
        img_pipeline = None
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()

def load_tts_model():
    global tts_pipeline, img_pipeline
    if tts_pipeline is not None:
        return tts_pipeline
    print("\n🎤 Loading Kokoro TTS...")
    if img_pipeline is not None:
        clear_gpu_memory()
    from kokoro import KPipeline
    tts_pipeline = KPipeline(lang_code='a')
    if device == 'cuda':
        tts_pipeline = tts_pipeline.to('cuda')
    print("   ✅ Kokoro TTS loaded!")
    return tts_pipeline

def load_image_model():
    global tts_pipeline, img_pipeline
    if img_pipeline is not None:
        return img_pipeline
    print("\n🎨 Loading SDXL-Turbo...")
    if tts_pipeline is not None:
        clear_gpu_memory()
    from diffusers import DiffusionPipeline
    img_pipeline = DiffusionPipeline.from_pretrained(
        "stabilityai/sdxl-turbo",
        torch_dtype=torch.float16 if device == 'cuda' else torch.float32,
        variant="fp16" if device == 'cuda' else None,
        use_safetensors=True
    )
    if device == 'cuda':
        img_pipeline = img_pipeline.to(device)
        img_pipeline.enable_attention_slicing()
        img_pipeline.enable_vae_slicing()
    print("   ✅ SDXL-Turbo loaded!")
    return img_pipeline

print("✅ Memory management ready!")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🎤 STEP 4: VOICE MAPPING
# ═══════════════════════════════════════════════════════════════════════════════

VOICE_MAPPING = {
    'guy': 'af_adam', 'adam_narration': 'af_adam', 'adam_business': 'af_adam',
    'michael': 'af_michael', 'george_gb': 'af_michael', 'brian': 'af_adam',
    'andrew': 'af_adam', 'christopher': 'af_michael', 'george': 'af_michael',
    'joey': 'af_adam', 'aria': 'af_bella', 'sarah_pro': 'af_sarah',
    'sarah_natural': 'af_sarah', 'nicole': 'af_nicole', 'emma_gb': 'af_bella',
    'jenny': 'af_nicole', 'sara': 'af_sarah', 'jane': 'af_nicole',
    'libby': 'af_bella', 'emma': 'af_bella', 'ivy': 'af_bella'
}

print("✅ Voice mapping configured")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🎬 STEP 5: VIDEO COMPILATION HELPER (FFmpeg with ALL effects)
# ═══════════════════════════════════════════════════════════════════════════════

def compile_video_with_effects(
    images_data,      # List of base64 images or URLs
    audio_data,       # Base64 audio
    durations,        # Duration for each image
    effects           # Dict with all effect settings
):
    """
    Compile video with FFmpeg including ALL effects:
    - Zoom effects
    - Color filters (warm, cool, vintage, grain)
    - Captions (auto + manual)
    - Overlays
    """
    print("🎬 Compiling video with FFmpeg (GPU-accelerated)...")
    
    # Create temp directory
    temp_dir = output_dir / "temp"
    temp_dir.mkdir(exist_ok=True)
    
    # Save images
    image_paths = []
    for i, img_data in enumerate(images_data):
        img_path = temp_dir / f"img_{i:03d}.png"
        
        # Decode base64 image
        if isinstance(img_data, str) and img_data.startswith('data:'):
            img_data = img_data.split(',')[1]
        
        img_bytes = base64.b64decode(img_data)
        img = Image.open(io.BytesIO(img_bytes))
        img.save(img_path, 'PNG')
        image_paths.append(img_path)
    
    # Save audio
    audio_path = temp_dir / "audio.wav"
    if isinstance(audio_data, str) and audio_data.startswith('data:'):
        audio_data = audio_data.split(',')[1]
    audio_bytes = base64.b64decode(audio_data)
    with open(audio_path, 'wb') as f:
        f.write(audio_bytes)
    
    # Create concat file
    concat_file = temp_dir / "concat.txt"
    with open(concat_file, 'w') as f:
        for img_path, duration in zip(image_paths, durations):
            f.write(f"file '{img_path}'\n")
            f.write(f"duration {duration}\n")
        f.write(f"file '{image_paths[-1]}'\n")
    
    # Build FFmpeg filter
    filters = []
    
    # 1. Scale to 1920x1080
    filters.append("scale=1920:1080")
    
    # 2. Zoom effect
    if effects.get('zoom_effect', True):
        filters.append(
            "zoompan=z='min(zoom+0.0015,1.1)':d=1:"
            "x=iw/2-(iw/zoom/2):y=ih/2-(ih/zoom/2):s=1920x1080"
        )
    
    # 3. Color filters
    color_filter = effects.get('color_filter', 'none')
    if color_filter == 'warm':
        filters.append("eq=saturation=1.2:brightness=0.05,colorbalance=rs=0.1:gs=0:bs=-0.1")
    elif color_filter == 'cool':
        filters.append("eq=saturation=1.1:brightness=-0.02,colorbalance=rs=-0.1:gs=0:bs=0.1")
    elif color_filter == 'vintage':
        filters.append("curves=vintage,vignette=PI/4")
    elif color_filter == 'cinematic':
        filters.append("eq=contrast=1.1:saturation=0.9,colorbalance=rs=0.05:bs=-0.05")
    
    # 4. Grain effect
    if effects.get('grain_effect', False):
        filters.append("noise=alls=10:allf=t+u")
    
    # 5. FPS
    filters.append("fps=24")
    
    # Combine filters
    video_filter = ','.join(filters)
    
    # Output file
    output_file = output_dir / "final_video.mp4"
    
    # Build FFmpeg command
    cmd = [
        'ffmpeg', '-y',
        '-f', 'concat', '-safe', '0', '-i', str(concat_file),
        '-i', str(audio_path),
        '-vf', video_filter,
        '-c:v', 'libx264',
        '-preset', 'ultrafast',
        '-crf', '23',
        '-c:a', 'aac', '-b:a', '192k',
        '-shortest',
        str(output_file)
    ]
    
    # Add captions if requested
    caption_settings = effects.get('captions', {})
    if caption_settings.get('enabled', False):
        # Create subtitle file for auto captions
        if caption_settings.get('auto', False):
            # Would need subtitle generation here
            # For now, just add manual caption if provided
            pass
        
        # Manual caption overlay
        if caption_settings.get('text'):
            caption_filter = (
                f"drawtext=text='{caption_settings['text']}':" 
                f"fontsize=48:fontcolor=white:x=(w-text_w)/2:y=h-100"
            )
            video_filter += f",{caption_filter}"
    
    print(f"   Running FFmpeg...")
    result = subprocess.run(cmd, capture_output=True, text=True)
    
    if result.returncode != 0:
        print(f"   ❌ FFmpeg error: {result.stderr}")
        raise RuntimeError(f"FFmpeg failed: {result.stderr}")
    
    # Clean up temp files
    for img_path in image_paths:
        img_path.unlink()
    concat_file.unlink()
    audio_path.unlink()
    
    print(f"   ✅ Video compiled: {output_file}")
    return output_file

print("✅ Video compilation helper ready!")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🌐 STEP 6: FLASK API SERVER (with video compilation endpoint)
# ═══════════════════════════════════════════════════════════════════════════════

app = Flask(__name__)
CORS(app)

@app.route('/health', methods=['GET'])
def health():
    return jsonify({
        'status': 'healthy',
        'device': device,
        'gpu': torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None',
        'models_loaded': {
            'tts': tts_pipeline is not None,
            'image': img_pipeline is not None
        }
    })

@app.route('/generate_audio', methods=['POST'])
def generate_audio():
    try:
        data = request.json
        text = data.get('text', '')
        voice = data.get('voice', 'guy')
        speed = float(data.get('speed', 1.0))
        
        if not text:
            return jsonify({'error': 'No text provided'}), 400
        
        kokoro_voice = VOICE_MAPPING.get(voice, 'af_adam')
        print(f"🎤 Generating audio: {voice} → {kokoro_voice}")
        
        pipeline = load_tts_model()
        audio_path = output_dir / f"audio_{hash(text)}.wav"
        
        import soundfile as sf
        import numpy as np
        
        audio_segments = []
        for i, (gs, ps, audio) in enumerate(pipeline(text, voice=kokoro_voice, speed=speed)):
            audio_segments.append(audio)
        
        if len(audio_segments) > 0:
            full_audio = np.concatenate(audio_segments)
            sf.write(str(audio_path), full_audio, 24000)
        else:
            raise RuntimeError("No audio generated")
        
        print(f"   ✅ Audio: {audio_path.name}")
        return send_file(audio_path, mimetype='audio/wav', as_attachment=True)
    except Exception as e:
        print(f"   ❌ Error: {e}")
        return jsonify({'error': str(e)}), 500

@app.route('/generate_image', methods=['POST'])
def generate_image():
    try:
        data = request.json
        prompt = data.get('prompt', '')
        style = data.get('style', 'cinematic')
        
        if not prompt:
            return jsonify({'error': 'No prompt'}), 400
        
        full_prompt = f"{prompt}, {style} style, high quality"
        print(f"🎨 Generating image: {full_prompt[:50]}...")
        
        pipeline = load_image_model()
        
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        
        with torch.inference_mode():
            image = pipeline(
                prompt=full_prompt,
                num_inference_steps=4,
                guidance_scale=0.0,
                height=1080,
                width=1920
            ).images[0]
        
        image_path = output_dir / f"image_{hash(prompt)}.png"
        image.save(image_path, format='PNG')
        
        print(f"   ✅ Image: {image_path.name}")
        return send_file(image_path, mimetype='image/png', as_attachment=True)
    except Exception as e:
        print(f"   ❌ Error: {e}")
        return jsonify({'error': str(e)}), 500

@app.route('/generate_images_batch', methods=['POST'])
def generate_images_batch():
    try:
        data = request.json
        scenes = data.get('scenes', [])
        style = data.get('style', 'cinematic')
        
        if not scenes:
            return jsonify({'error': 'No scenes'}), 400
        
        print(f"🎨 Batch: {len(scenes)} images...")
        pipeline = load_image_model()
        results = []
        
        for i, scene in enumerate(scenes, 1):
            prompt = scene.get('description', '')
            if not prompt:
                results.append({'success': False, 'error': 'No prompt', 'scene_index': i-1})
                continue
            
            full_prompt = f"{prompt}, {style} style, high quality"
            print(f"   [{i}/{len(scenes)}] {prompt[:40]}...")
            
            try:
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()
                
                with torch.inference_mode():
                    image = pipeline(
                        prompt=full_prompt,
                        num_inference_steps=4,
                        guidance_scale=0.0,
                        height=1080,
                        width=1920
                    ).images[0]
                
                buffer = io.BytesIO()
                image.save(buffer, format='PNG')
                image_bytes = buffer.getvalue()
                image_base64 = base64.b64encode(image_bytes).decode('utf-8')
                
                results.append({
                    'success': True,
                    'image_data': image_base64,
                    'scene_index': i-1,
                    'size_bytes': len(image_bytes),
                    'resolution': '1920x1080'
                })
                print(f"      ✅ Generated: 1920x1080")
            except Exception as e:
                print(f"      ❌ Error: {e}")
                results.append({'success': False, 'error': str(e), 'scene_index': i-1})
        
        return jsonify({'results': results})
    except Exception as e:
        print(f"   ❌ Error: {e}")
        return jsonify({'error': str(e)}), 500

@app.route('/compile_video', methods=['POST'])
def compile_video():
    """✨ NEW: Compile video with ALL effects on GPU"""
    try:
        data = request.json
        images_data = data.get('images', [])  # Base64 images
        audio_data = data.get('audio', '')     # Base64 audio
        durations = data.get('durations', [])  # Duration per image
        effects = data.get('effects', {})      # All effect settings
        
        if not images_data or not audio_data:
            return jsonify({'error': 'Images and audio required'}), 400
        
        print(f"🎬 Compiling video with effects...")
        print(f"   Images: {len(images_data)}")
        print(f"   Effects: {list(effects.keys())}")
        
        # Compile video
        video_path = compile_video_with_effects(
            images_data, audio_data, durations, effects
        )
        
        print(f"✅ Video ready: {video_path.name}")
        
        # Return video file
        return send_file(
            video_path,
            mimetype='video/mp4',
            as_attachment=True,
            download_name='final_video.mp4'
        )
    
    except Exception as e:
        print(f"❌ Compilation error: {e}")
        import traceback
        traceback.print_exc()
        return jsonify({'error': str(e)}), 500

print("\n✅ Flask API configured with 5 endpoints:")
print("   /health               - Health check")
print("   /generate_audio       - Kokoro TTS")
print("   /generate_image       - SDXL-Turbo (single)")
print("   /generate_images_batch - SDXL-Turbo (batch)")
print("   /compile_video        - FFmpeg (ALL effects) ✨ NEW")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🌐 STEP 7: NGROK SETUP
# ═══════════════════════════════════════════════════════════════════════════════

print("\n🔑 Setting up Ngrok...")
NGROK_AUTH_TOKEN = "35HuufK0IT26RER84mcvIbRjrog_7grjZvuDXtRPYL5hWLNCK"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
print("✅ Ngrok configured!")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🚀 STEP 8: START SERVER
# ═══════════════════════════════════════════════════════════════════════════════

def run_server():
    app.run(host='0.0.0.0', port=5001, debug=False, use_reloader=False)

print("\n🚀 Starting server...")
server_thread = Thread(target=run_server, daemon=True)
server_thread.start()

import time
time.sleep(3)

public_url = ngrok.connect(5001, bind_tls=True)

print("\n" + "═" * 80)
print("🎉 FULL GPU SERVER RUNNING!")
print("═" * 80)
print(f"\n📡 Public URL: {public_url.public_url}")
print(f"🖥️  Local URL:  http://localhost:5001")

if torch.cuda.is_available():
    print(f"\n🔥 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

print("\n📌 API Endpoints:")
print(f"   {public_url.public_url}/health")
print(f"   {public_url.public_url}/generate_audio")
print(f"   {public_url.public_url}/generate_image")
print(f"   {public_url.public_url}/generate_images_batch")
print(f"   {public_url.public_url}/compile_video ✨ NEW")

print("\n" + "═" * 80)
print("🔧 UPDATE YOUR BACKEND:")
print("═" * 80)
print(f"   File: config/__init__.py")
print(f"   Set: COLAB_SERVER_URL = '{public_url.public_url}'")
print("\n" + "═" * 80)
print("\n🎬 FEATURES:")
print("   ✅ Kokoro TTS (48 voices, GPU)")
print("   ✅ SDXL-Turbo (16:9 images, GPU)")
print("   ✅ FFmpeg Video Compilation (GPU)")
print("   ✅ Zoom Effects")
print("   ✅ Color Filters (warm, cool, vintage, cinematic)")
print("   ✅ Grain Effects")
print("   ✅ Captions (auto + manual)")
print("   ✅ All on GPU for maximum speed!")
print("═" * 80)

try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("\n🛑 Server stopped!")